Part2 - Apriori Algorithm

In [31]:
import numpy as np
from itertools import combinations
import pandas as pd

In [32]:
file_name = "associationruletestdata"

def getData(file_name):
    file_handle = open(file_name + ".txt", "r")
    lines = file_handle.readlines()
    input_matrix = []
    disease_matrix = []
    transactions = []
    for line in lines:
        all_list = line.strip().split("\t")
        updown_list = []
        disease=[]
        j = 1
        for expression in all_list:
            if j == len(all_list):
                disease.append(expression)
                updown_list.append(expression)
            else:
                updown_list.append("G" + str(j) + "_" + expression)
            j += 1
        input_matrix.append(updown_list)
        disease_matrix.append(disease)
    return input_matrix, disease_matrix

In [33]:
def getInitialData(input_matrix,support):
    count_dict = {}
    accepted_list = []
    for row in range(0, len(input_matrix)-1):
        for col in range(0, len(input_matrix[0])):
            if not input_matrix[row][col] in count_dict:
                count_dict[input_matrix[row][col]] = 1
            else:
                count_dict[input_matrix[row][col]] += 1
    for key in count_dict.keys():
        accepted_list.append(key)
    return count_dict, accepted_list

In [34]:
def findFreqItemSets(globalSupDict,input_matrix, accepted_list, support):
    countAllItemsets = 0#len(accepted_list)
    globalSupDict={}
    final_accepted_list = []
    for i in range(1, len(input_matrix[0])-1):
        count_dict = {}
        combinations_list = list(combinations(set(accepted_list), i))
        for row in range(len(input_matrix)):
            for tuples in combinations_list:
                if(all(elm in input_matrix[row] for elm in list(tuples))):
                    if not tuples in count_dict:
                        count_dict[tuples] = 1
                    else:
                        count_dict[tuples] += 1
        count=0;
        accepted_list=[]
        for key in count_dict:
            if (count_dict.get(key)/(len(input_matrix))) >= (support/100.0):
                count+=1
                globalSupDict[tuple(sorted(key))] = count_dict[key]#/(len(input_matrix))
                if(key not in final_accepted_list):
                    final_accepted_list.append(list(key))
                for j in key:
                    accepted_list.append(j)
        countAllItemsets+=count
        if(len(accepted_list)<1):
            print("Number of all lengths frequent itemsets: " + str(countAllItemsets))
            return final_accepted_list, globalSupDict
        print("Number of length-"+str(i)+" frequent itemsets: " + str(count))

In [35]:
def generateRules(globalSupDict, min_conf):
    
    headlst = []
    bodylst = []
    rules = []
    keys = globalSupDict.keys()
    count = 0
   
    for key in keys:
        for s1 in range(1, len(key)):
            comb = set(combinations(key,s1))
            for item in comb:
                confidence = globalSupDict.get(key)/globalSupDict.get(item)
                if confidence >= min_conf/100.0:
                    head = list(key)
                    head1 = []
                    for x in head:
                        if(x not in item):
                            head1.append(x)
                    headlst.append(list(head1))
                    bodylst.append(list(item))
                    count = count+1      
                    rules.append(str(list(head1))+"-->"+str(list(item)))
    
    return headlst, bodylst, rules, count

In [36]:
def template1(p1,p2,p3): 
    count = 0
    outputList = []
    allfreqs1 = []
    if(p1.upper() == "BODY"):
        if(p2.upper()=="ANY"):
            for i in range(0,len(bodies)):
                if(set(p3).issubset(set(bodies[i]))):
                    outputList.append(i)
                    allfreqs1.append(bodies[i])
                    count = count+1
        elif(p2.upper()=="NONE"):
            for i in range(0,len(bodies)):
                if(len(set(p3).intersection(set(bodies[i])))==0):
                    outputList.append(i)
                    allfreqs1.append(bodies[i])
                    count = count+1
        else:
            for i in range(0,len(bodies)):
                if(len(set(p3).intersection(set(bodies[i])))==1):
                    outputList.append(i)
                    allfreqs1.append(bodies[i])
                    count = count+1
            
    elif(p1.upper() == "HEAD"):
        if(p2.upper()=="ANY"):
            for i in range(0,len(heads)):
                if(set(p3).issubset(set(heads[i]))):
                    outputList.append(i)
                    allfreqs1.append(heads[i])
                    count = count+1
        elif(p2.upper()=="NONE"):
            for i in range(0,len(heads)):
                if(len(set(p3).intersection(set(heads[i])))==0):
                    outputList.append(i)
                    allfreqs1.append(heads[i])
                    count = count+1
        else:
            for i in range(0,len(heads)):
                if(len(set(p3).intersection(set(heads[i])))==1):
                    allfreqs1.append(heads[i])
                    outputList.append(i)
                    count = count+1
            
    else:
        if(p2.upper()=="ANY"):
            for i in range(len(bodies)):
                if(len(set(p3).intersection(set(bodies[i]))) == 1 or len(set(p3).intersection(set(heads[i])))) == 1:
                    #outputList.append(i)
                    allfreqs1.append(str(heads[i])+"-->"+str(bodies[i]))
                    count = count+1

        elif(p2.upper()=="NONE"):
            for i in range(len(bodies)):
                if(len(set(p3).intersection(set(bodies[i])))) == 0:
                    if(len(set(p3).intersection(set(heads[i])))) == 0:
                        #allfreqs.append(heads[i])
                        allfreqs1.append(str(heads[i])+"-->"+str(bodies[i]))
                        count = count+1
        else:
            for i in range(len(bodies)):
                if len(set(p3).intersection(set(bodies[i]))) == 1 or len(set(p3).intersection(set(heads[i]))) == 1:
                    if not(len(set(p3).intersection(set(bodies[i]))) == 1 and len(set(p3).intersection(set(heads[i])))) == 1:
                        allfreqs1.append(str(heads[i])+"-->"+str(bodies[i]))
                        count = count +1
    return allfreqs1, outputList, count

In [37]:
def template2(par1,par2):
    count=0
    par2 = int(par2)
    outputList = []
    allfreqs2 = []
    if(par1.upper()=="HEAD"):
        for i in range(0, len(heads)):
            if(len(heads[i])>=par2):
                outputList.append(i)
                allfreqs2.append(heads[i])
                count = count+1
    elif(par1.upper()=="BODY"):
        for i in range(0,len(bodies)):
            if(len(bodies[i])>=par2):
                outputList.append(i)
                allfreqs2.append(bodies[i])
                count = count+1
    else:
        for i in range(len(heads)):
            if(len(heads[i])+len(bodies[i])>=par2):
                allfreqs2.append(str(heads[i])+"-->"+str(bodies[i]))
                count = count+1
        
    return allfreqs2, outputList, count

In [38]:
def template3(par1):
    parameters = par1.split("+")
    overallList = []
    op = parameters[0]
    final_list=[]
    
    if(op[1:-1].lower()=="or"):
        
        if(op[0]=="1"):
            allfre1, output1, count=template1(parameters[1],parameters[2],parameters[3].split(","))

        elif(op[0]=="2"):
            allfre1, output1, count=template2(parameters[1],parameters[2])
            
        overallList.append(output1)
        
        if(op[-1]=="1"):
            allfre1, output1, count=template1(parameters[-3:][0],parameters[-3:][1],parameters[-3:][2].split(","))
        elif(op[-1]=="2"):
            allfre1, output1, count=template2(parameters[-2:][0],parameters[-2:][1])
            
        overallList.append(output1)
        
        final_list = list(set(overallList[0]).union(set(overallList[1])))
    
    elif(op[1:-1].lower()=="and"):
        
        if(op[0]=="1"):
            allfre1, output1, count=template1(parameters[1],parameters[2],parameters[3].split(","))
        elif(op[0]=="2"):
            allfre1, output1, count=template2(parameters[1],parameters[2])
        
        overallList.append(output1)
        
        if(op[-1]=="1"):
            allfre1, output1, count=template1(parameters[-3:][0],parameters[-3:][1],parameters[-3:][2].split(","))
        elif(op[-1]=="2"):
            allfre1, output1, count=template2(parameters[-2:][0],parameters[-2:][1])
        
        overallList.append(output1)
        
        final_list = [value for value in overallList[0] if value in overallList[1]] 
    
    count = len(final_list)
    return final_list, count

Driver Code:

Run this to generate all frequent item sets

In [39]:
support = 50
min_conf = 70

input_matrix, disease_matrix = getData(file_name)
count_Dict, accepted_list = getInitialData(input_matrix, support)
accepted_list, globalSupDict=findFreqItemSets(count_Dict,input_matrix, accepted_list, support)

Number of length-1 frequent itemsets: 109
Number of length-2 frequent itemsets: 63
Number of length-3 frequent itemsets: 2
Number of all lengths frequent itemsets: 174


##### Run this to generate rules of association

In [40]:
heads, bodies, rules, numRules = generateRules(globalSupDict, min_conf)
print("Number of rules generated: "+str(numRules))
# print("Rules:")
# for i in rules:
#     print(i)

Number of rules generated: 117


In [44]:
heads

[['G97_Down'],
 ['G82_Down'],
 ['G72_Up'],
 ['G59_Up'],
 ['G13_Down'],
 ['G59_Up'],
 ['G82_Down'],
 ['G59_Up'],
 ['G59_Up'],
 ['G72_Up'],
 ['G59_Up'],
 ['G96_Down'],
 ['G59_Up'],
 ['G59_Up'],
 ['G82_Down'],
 ['G13_Down'],
 ['G72_Up'],
 ['G13_Down'],
 ['G28_Down'],
 ['G13_Down'],
 ['G6_Up'],
 ['G13_Down'],
 ['G72_Up'],
 ['G82_Down'],
 ['G82_Down'],
 ['G96_Down'],
 ['G72_Up'],
 ['G1_Up'],
 ['G72_Up'],
 ['G96_Down'],
 ['G52_Down'],
 ['G28_Down'],
 ['G6_Up'],
 ['G28_Down'],
 ['G59_Up'],
 ['G88_Down'],
 ['G87_Up'],
 ['G88_Down'],
 ['G38_Down'],
 ['G88_Down'],
 ['G54_Up'],
 ['G59_Up'],
 ['G59_Up'],
 ['G1_Up'],
 ['G38_Down'],
 ['G54_Up'],
 ['G1_Up'],
 ['G38_Down'],
 ['G38_Down'],
 ['G52_Down'],
 ['G38_Down'],
 ['G47_Up'],
 ['G38_Down'],
 ['G59_Up'],
 ['G54_Up'],
 ['G24_Down'],
 ['G72_Up'],
 ['G28_Down'],
 ['G2_Down'],
 ['G28_Down'],
 ['G32_Down'],
 ['G87_Up'],
 ['G28_Down'],
 ['G6_Up'],
 ['G32_Down'],
 ['G38_Down'],
 ['G28_Down'],
 ['G38_Down'],
 ['G47_Up'],
 ['G28_Down'],
 ['G38_Down'],
 ['G

Template 1:

In [43]:
par1 = "body"
par2 = "none"
par3 = "G59_Up"
allfreqitems1, outputList1, count1=template1(par1,par2,par3.split(","))
print((str(count1))+" rules found!")
print("Itemsets:")
for i in allfreqitems1:
    print(i)

108 rules found!
Itemsets:
['G82_Down']
['G97_Down']
['G97_Down']
['G13_Down']
['G82_Down']
['G1_Up']
['G72_Up']
['G96_Down']
['G28_Down']
['G6_Up']
['G13_Down']
['G82_Down']
['G13_Down']
['G72_Up']
['G13_Down']
['G28_Down']
['G13_Down']
['G6_Up']
['G82_Down']
['G72_Up']
['G96_Down']
['G82_Down']
['G1_Up']
['G72_Up']
['G96_Down']
['G72_Up']
['G28_Down']
['G52_Down']
['G28_Down']
['G6_Up']
['G88_Down']
['G88_Down']
['G87_Up']
['G88_Down']
['G54_Up']
['G88_Down']
['G87_Up']
['G38_Down']
['G38_Down']
['G1_Up']
['G1_Up']
['G54_Up']
['G2_Down']
['G65_Down']
['G38_Down']
['G52_Down']
['G38_Down']
['G47_Up']
['G32_Down']
['G24_Down']
['G54_Up']
['G32_Down']
['G2_Down']
['G28_Down']
['G32_Down']
['G28_Down']
['G28_Down']
['G87_Up']
['G32_Down']
['G6_Up']
['G94_Up']
['G38_Down']
['G28_Down']
['G28_Down']
['G47_Up']
['G32_Down']
['G38_Down']
['G38_Down']
['G91_Up']
['G1_Up']
['G70_Down']
['G10_Down']
['G70_Down']
['G8_Up']
['G88_Down']
['G24_Down']
['G88_Down']
['G28_Down']
['G88_Down']
['G88_Do

Template 2:

In [45]:
par1 = "body"
par2 = "2"
allfreqitems2, outputList2, count2=template2(par1,par2)
print((str(count2))+" rules found!")

6 rules found!


Template 3:

In [23]:
par1 = "1and1+head+any+G1_Up+body+1_G59_Up"
outputList3, count3=template3(par1)
print((str(count3))+" rules found!")

0 rules found!


Run this to generate results according to templates:

In [24]:
i=0
while(i==0):
    choice = input("Enter template choice 1, 2, 3: ")
    final_count = 0
    try:
        if(choice == "1"):
            par1 = input("Enter first parameter: ")
            par2 = input("Enter second parameter: ")
            par3 = input("Enter third paramter: ")
            allfreq1, outputList2, count=template1(par1,par2,par3.split(","))
            final_count = count
            print((str(final_count))+" rules found!")
        
        elif(choice == "2"):
            par1 = input("Enter first parameter: ")
            par2 = input("Enter second parameter: ")
            allfreq2, outputList2, count=template2(par1,par2)
            final_count = count
            print((str(final_count))+" rules found!")
            
        elif(choice == "3"):
            count = 0
            query = ""
            par1 = input("Enter first parameter: ")
            if(par1[0]=="1"):
                print("Enter parameters for 1st template: ")
                par2 = input("Enter first parameter: ")
                par3 = input("Enter second parameter: ")
                par4 = input("Enter third paramter: ")
                query += par1+"+"+par2+"+"+par3+"+"+par4
            elif(par1[0]=="2"):
                print("Enter parameters for 2nd template: ")
                par2 = input("Enter first parameter: ")
                par3 = input("Enter second parameter: ")
                query += par1+"+"+par2+"+"+par3
            if(par1[-1] == "1"):
                print("Enter parameters for 1st template: ")
                par1 = input("Enter first parameter: ")
                par2 = input("Enter second parameter: ")
                par3 = input("Enter third paramter: ")
                query += "+"+par1+"+"+par2+"+"+par3
            elif(par1[-1]=="2"):
                print("Enter parameters for 2nd template: ")
                par1 = input("Enter first parameter: ")
                par2 = input("Enter second parameter: ")
                query += "+"+par1+"+"+par2
            outputList2, count=template3(query)
            final_count = count
            print((str(final_count))+" rules found!")
        else:
            print("Program Exiting!")  
            break
    except Exception as e:
        print(e)
        print("Invalid template choice. Please run the program again.")
        
        
        
        


KeyboardInterrupt: 